In [1]:
from pathlib import Path
import shutil
from typing import Dict
from tqdm.notebook import tqdm
import numpy as np

In [2]:
input_dir = Path("D://2022-05-20_carysfort_FS003_260")
output_dir = Path("D://2022-05-20_carysfort_FS003_260_data")
label_dir = Path("D://2022-05-20_carysfort_FS003_260_labels")
max_permissible_difference_s = 0.1

In [3]:
color_frame_t: Dict[float, Path] = {}
for color_frame in tqdm(input_dir.glob('*_Color_t[0-9.]*')):
    fname = color_frame.stem
    t = float(fname[fname.find('_t') + 2:])
    color_frame_t[t] = color_frame

0it [00:00, ?it/s]

In [4]:
color_times = list(color_frame_t.keys())

In [5]:
depth_frame_t: Dict[float, Path] = {}
for depth_frame in tqdm(input_dir.glob('*_Depth_t[0-9.]*')):
    fname = depth_frame.stem
    t = float(fname[fname.find('_t') + 2:])
    depth_frame_t[t] = depth_frame

0it [00:00, ?it/s]

In [6]:
depth_times = list(depth_frame_t.keys())

In [7]:
rgb_idx = 0
depth_idx = 0
frame_idx = 0
with tqdm(total=len(depth_times)) as pbar:
    while rgb_idx < len(color_times) and depth_idx < len(depth_times):
        initial_difference_s = color_times[rgb_idx] - depth_times[depth_idx]
        if initial_difference_s < -1 * max_permissible_difference_s:
            rgb_idx += 1
        elif initial_difference_s >  max_permissible_difference_s:
            depth_idx += 1
            pbar.update(1)
        else:
            depth_time = depth_times[depth_idx]
            color_time = color_times[rgb_idx]

            depth_files = input_dir.glob(f"*Depth*_t{depth_time:.9f}*")
            color_files = input_dir.glob(f"*Color*_t{color_time:.9f}*")

            frame_folder = output_dir.joinpath(f'frame_{frame_idx:06d}')
            frame_folder.mkdir(exist_ok=True, parents=True)
            for depth_file in depth_files:
                shutil.move(depth_file, frame_folder.joinpath(depth_file.name))
            for color_file in color_files:
                if color_file.suffix.endswith('png'):
                    shutil.copy(color_file, label_dir.joinpath(color_file.name))
                shutil.move(color_file, frame_folder.joinpath(color_file.name))
            rgb_idx += 1
            depth_idx += 1
            frame_idx += 1
            pbar.update(1)


  0%|          | 0/77 [00:00<?, ?it/s]

In [8]:
if False:
    for frame_folder in output_dir.glob('frame*'):
        for frame_file in frame_folder.glob('*'):
            shutil.move(frame_file, input_dir.joinpath(frame_file.name))
        shutil.rmtree(frame_folder)